In [1]:
import numpy as np
import cv2
from scipy.optimize import least_squares
import os
import open3d as o3d
import copy
from tqdm import tqdm

import plotly.graph_objects as go
fig = go.Figure()

## Find Features with SIFT and Lowe's Filter

In this step, we use the **SIFT (Scale-Invariant Feature Transform)** algorithm to detect features in images. Following that, we employ a **Brute-Force KNN Matcher** with Lowe's filter to identify strong matches between the detected features.


In [2]:
def find_features(img0, img1):
    """
    Detect keypoints and compute descriptors for two images using the SIFT algorithm.

    Parameters:
    - img0: First input image.
    - img1: Second input image.

    Returns:
    - pts0: Keypoints in the first image.
    - pts1: Keypoints in the second image.
    """
    # Convert images to grayscale
    img0gray = cv2.cvtColor(img0, cv2.COLOR_BGR2GRAY)
    img1gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)

    # Initialize SIFT detector
    sift = cv2.SIFT_create()

    # Detect keypoints and compute descriptors for both images
    kp0, des0 = sift.detectAndCompute(img0gray, None)
    kp1, des1 = sift.detectAndCompute(img1gray, None)

    # Use Brute-Force Matcher for descriptor matching
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des0, des1, k=2)

    # Apply ratio test to find good matches
    good = []
    for m, n in matches:
        if m.distance < 0.70 * n.distance:
            good.append(m)

    # Extract coordinates of keypoints from good matches
    pts0 = np.float32([kp0[m.queryIdx].pt for m in good])
    pts1 = np.float32([kp1[m.trainIdx].pt for m in good])
    return pts0, pts1

## Data Association for 3D Reconstruction

This code performs data association between initial points (pts1) and new points (pts2 and pts3). The primary goal is to find common points between two sets of points from different images.


In [3]:
def data_association(pts1, pts2, pts3):
    """
    Perform data association between initial points and new points.

    Find common points between two sets of points (pts1 and pts2) and a third set (pts3).

    Parameters:
    - pts1: Points from the first image.
    - pts2: Points from the second image.
    - pts3: Points from the third image.

    Returns:
    - common_indices_1: Indices of common points in pts1.
    - common_indices_2: Indices of common points in pts2.
    - unmatched_pts2: Points in pts2 that are not common.
    - unmatched_pts3: Points in pts3 that are not common.
    """
    if pts1.shape[0] == 0 or pts2.shape[0] == 0 or pts3.shape[0] == 0:
        print("One or more sets of points are empty.")
        return None, None, None, None

    common_indices_1 = []
    common_indices_2 = []

    # Iterate through points in pts1
    for i in range(pts1.shape[0]):
        matching_indices = np.where((pts2 == pts1[i, :]).all(axis=1))
        # If a match is found in pts2
        if matching_indices[0].size > 0:
            common_indices_1.append(i)
            common_indices_2.append(matching_indices[0][0])

    if not common_indices_1 or not common_indices_2:
        print("No common points found.")
        return None, None, None, None

    # Create masked arrays to filter out common points
    unmatched_pts2 = np.ma.array(pts2, mask=False)
    unmatched_pts2.mask[common_indices_2] = True
    unmatched_pts2 = unmatched_pts2.compressed()
    unmatched_pts2 = unmatched_pts2.reshape(int(unmatched_pts2.shape[0] / 2), 2)

    unmatched_pts3 = np.ma.array(pts3, mask=False)
    unmatched_pts3.mask[common_indices_2] = True
    unmatched_pts3 = unmatched_pts3.compressed()
    unmatched_pts3 = unmatched_pts3.reshape(int(unmatched_pts3.shape[0] / 2), 2)

    return np.array(common_indices_1), np.array(common_indices_2), unmatched_pts2, unmatched_pts3

## Triangulation for 3D Reconstruction

This function performs triangulation to reconstruct 3D points from corresponding 2D points in two images.


In [4]:
def Triangulation(P1, P2, pts1, pts2):
    """
    Perform triangulation to reconstruct 3D points from corresponding 2D points in two images.

    Parameters:
    - P1: Projection matrix for the first camera.
    - P2: Projection matrix for the second camera.
    - pts1: 2D points in the first image.
    - pts2: 2D points in the second image.

    Returns:
    - points1: Transposed 2D points in the first image.
    - points2: Transposed 2D points in the second image.
    - cloud: Triangulated 3D points.
    """
    # Transpose 2D point arrays to get input data in the expected format for cv2.triangulatePoints
    points1 = np.transpose(pts1)
    points2 = np.transpose(pts2)

    # Perform triangulation to obtain homogeneous 3D coordinates
    cloud = cv2.triangulatePoints(P1, P2, points1, points2)
    
    # Convert homogeneous coordinates to non-homogeneous coordinates
    cloud = cloud / cloud[3]

    return points1, points2, cloud

## Perspective-n-Point (PnP) Pose Estimation

This function estimates the camera pose using the Perspective-n-Point (PnP) algorithm, specifically using `cv2.solvePnPRansac`.


In [5]:
def PnP(world_points, image_points, intrinsic_matrix, distortion_coefficients, original_image_points, initial):
    """
    Estimate camera pose using solvePnPRansac.

    Parameters:
    - world_points: 3D coordinates of world points.
    - image_points: 2D coordinates of corresponding image points.
    - intrinsic_matrix: Intrinsic camera matrix.
    - distortion_coefficients: Distortion coefficients.
    - original_image_points: Original 2D image points.
    - initial: Flag indicating whether the function is used in the initial stage.

    Returns:
    - rotation_matrix: Rotation matrix.
    - translation_vector: Translation vector.
    - filtered_image_points: Filtered 2D image points.
    - filtered_world_points: Filtered 3D world points.
    - filtered_original_image_points: Filtered original 2D image points.
    """
    # Check if the function is used in the initial stage
    if initial == 1:
        # If initial, reshape the input arrays for consistency
        world_points = world_points[:, 0, :]  # to remove empty dimension
        image_points = image_points.T
        original_image_points = original_image_points.T

    # Check if there are enough 3D points for PnP
    if len(world_points) < 4:
        print("Not enough points for PnP.")
        return None, None, None, world_points, original_image_points

    # Use solvePnPRansac to estimate the pose (R, t) from 3D-2D correspondences
    ret, rotation_vector, translation_vector, inliers = cv2.solvePnPRansac(
        world_points, image_points, intrinsic_matrix, distortion_coefficients, cv2.SOLVEPNP_ITERATIVE
    )

    # Check if the PnP solution is successful
    if ret is not None:
        # Convert rotation vector to rotation matrix
        rotation_matrix, _ = cv2.Rodrigues(rotation_vector)

        # If inliers are obtained, filter the input arrays accordingly
        if inliers is not None:
            image_points = image_points[inliers[:, 0]]
            world_points = world_points[inliers[:, 0]]
            original_image_points = original_image_points[inliers[:, 0]]

        # Return the rotation matrix (R), translation vector (t), filtered 2D points (image_points),
        # filtered 3D points (world_points), and filtered original 2D points (original_image_points)
        return rotation_matrix, translation_vector, image_points, world_points, original_image_points
    else:
        print("PnP failed to find a solution.")
        # If PnP fails, return None for rotation matrix, translation vector, and filtered arrays
        return None, None, None, world_points, original_image_points

## Reprojection Error for optimization

This function computes the reprojection error of 3D world points onto 2D image points, providing a metric for the accuracy of the camera pose.


In [6]:
def compute_reprojection_error(world_points, image_points, camera_pose, intrinsic_matrix, homogeneity):
    """
    Compute the reprojection error of 3D world points onto 2D image points.

    Parameters:
    - world_points: 3D coordinates of world points.
    - image_points: 2D coordinates of corresponding image points.
    - camera_pose: Camera pose matrix (extrinsic parameters).
    - intrinsic_matrix: Intrinsic camera matrix.
    - homogeneity: Flag indicating whether to use homogeneous coordinates.

    Returns:
    - total_error: Total reprojection error.
    - transformed_world_points: Transformed world points.
    - projected_points: Projected points on the image.
    """
    total_error = 0

    # Extract rotation and translation from the camera pose matrix
    rotation_matrix = camera_pose[:3, :3]
    translation_vector = camera_pose[:3, 3]

    # Convert rotation matrix to Rodrigues representation
    rvec, _ = cv2.Rodrigues(rotation_matrix)

    # Convert 3D world points to homogeneous coordinates if required
    if homogeneity == 1:
        world_points = cv2.convertPointsFromHomogeneous(world_points.T)

    # Project 3D world points onto the 2D image plane
    projected_points, _ = cv2.projectPoints(world_points, rvec, translation_vector, intrinsic_matrix, distCoeffs=None)
    projected_points = projected_points[:, 0, :]
    projected_points = np.float32(projected_points)

    # Compute reprojection error
    if homogeneity == 1:
        total_error = cv2.norm(projected_points, image_points.T, cv2.NORM_L2)
    else:
        total_error = cv2.norm(projected_points, image_points, cv2.NORM_L2)

    # Transpose image_points for further processing
    image_points = image_points.T

    # Normalize total error by the number of points
    normalized_error = total_error / len(projected_points)

    return normalized_error, world_points, projected_points


## Bundle Adjustment

This function performs bundle adjustment to optimize the parameters, including projection matrix, intrinsic matrix, image points, and 3D world points.


In [7]:
#  https://scipy-cookbook.readthedocs.io/items/bundle_adjustment.html

def calculate_optimization_reprojection_error(params):
    # Extracting parameters
    projection_matrix = params[0:12].reshape((3, 4))
    intrinsic_matrix = params[12:21].reshape((3, 3))
    
    # Extracting image points
    num_image_points = len(params[21:])
    num_image_points_to_use = int(num_image_points * 0.4)
    image_points = params[21:21 + num_image_points_to_use].reshape((2, int(num_image_points_to_use / 2)))
    
    # Extracting 3D points
    num_3d_points = int(len(params[21 + num_image_points_to_use:]) / 3)
    three_d_points = params[21 + num_image_points_to_use:].reshape((num_3d_points, 3))
    
    # Extracting rotation and translation
    rotation_matrix = projection_matrix[:3, :3]
    translation_vector = projection_matrix[:3, 3]

    image_points = image_points.T
    num_points = len(image_points)
    error = []
    
    rotation_vector, _ = cv2.Rodrigues(rotation_matrix)
    
    projected_2d_points, _ = cv2.projectPoints(three_d_points, rotation_vector, translation_vector, intrinsic_matrix, distCoeffs=None)
    projected_2d_points = projected_2d_points[:, 0, :]
    
    for idx in range(num_points):
        actual_image_point = image_points[idx]
        reprojected_image_point = projected_2d_points[idx]
        point_error = (actual_image_point - reprojected_image_point) ** 2
        error.append(point_error)

    error_array = np.array(error).ravel() / num_points

    return error_array

def bundle_adjustment(points_3d, image_points, projection_matrix, intrinsic_matrix, reprojection_error_threshold):
    # Concatenate parameters for optimization
    optimization_variables = np.hstack((projection_matrix.ravel(), intrinsic_matrix.ravel()))
    optimization_variables = np.hstack((optimization_variables, image_points.ravel()))
    optimization_variables = np.hstack((optimization_variables, points_3d.ravel()))

    # Calculate initial error
    initial_error = np.sum(calculate_optimization_reprojection_error(optimization_variables))

    # Use least squares optimization to minimize reprojection error
    optimized_values = least_squares(fun=calculate_optimization_reprojection_error, x0=optimization_variables, gtol=reprojection_error_threshold)

    # Extract optimized values
    optimized_values = optimized_values.x
    optimized_projection_matrix = optimized_values[:12].reshape((3, 4))

    remaining_elements = len(optimized_values[21:])
    num_image_points = int(remaining_elements * 0.4)
    optimized_image_points = optimized_values[21:21 + num_image_points].reshape((2, num_image_points // 2)).T

    optimized_world_coordinates = optimized_values[21 + num_image_points:].reshape((int(remaining_elements / 3), 3))
    optimized_image_points = optimized_image_points.T

    return optimized_world_coordinates, optimized_image_points, optimized_projection_matrix

In [8]:
def draw_points(image, pts, repro):
    if repro == False:
        image = cv2.drawKeypoints(image, pts, image, color=(0, 255, 0), flags=0)
    else:
        for p in pts:
            image = cv2.circle(image, tuple(p), 2, (0, 0, 255), -1)
    return image

In [9]:
def create_pointcloud(output_path, point_cloud, colors):
    """
    Convert 3D point cloud data to PLY format and save it to a file.

    Parameters:
    - output_path: Path to the directory where the PLY file will be saved.
    - point_cloud: 3D point cloud data (numpy array).
    - colors: RGB colors corresponding to each point in the cloud.

    Returns:
    None
    """
    # Rescale point cloud coordinates and concatenate with colors
    scaled_points = point_cloud.reshape(-1, 3) * 200
    colored_points = np.hstack([scaled_points, colors.reshape(-1, 3)])

    # Clean the point cloud by removing points outside a certain distance from the mean
    mean_point = np.mean(colored_points[:, :3], axis=0)
    centered_points = colored_points[:, :3] - mean_point
    distances = np.sqrt(centered_points[:, 0] ** 2 + centered_points[:, 1] ** 2 + centered_points[:, 2] ** 2)
    valid_indices = np.where(distances < np.mean(distances) + 300)
    cleaned_points = colored_points[valid_indices]

    # Define PLY header
    ply_header = '''ply
        format ascii 1.0
        element vertex %(num_verts)d
        property float x
        property float y
        property float z
        property uchar blue
        property uchar green
        property uchar red
        end_header
        '''

    # Write the PLY file
    with open(os.path.join(output_path, "Point_Cloud", "sparse_ptcloud1.ply"), "w") as file:
        file.write(ply_header % dict(num_verts=len(cleaned_points)))
        np.savetxt(file, cleaned_points, '%f %f %f %d %d %d')


## SFM pipeline
The SFM Pipeline is designed to perform incremental Structure from Motion (SFM) for multiple images

In [10]:
def sfm_pipeline(path, img_dir, intrinsic_matrix, perform_bundle_adjustment=False):
    image_dataset = os.path.basename(img_dir)

    # Initialize variables and parameters
    print('Initialized Initial Parameters')
    cv2.namedWindow('image', cv2.WINDOW_NORMAL)
    pose_array = intrinsic_matrix.ravel()

    # Initialize Pose at a (4 x 4) Identity Matrix
    initial_pose_matrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0]])                
    current_pose_matrix = np.empty((3, 4))

    projection_matrix_1 = np.matmul(intrinsic_matrix, initial_pose_matrix)
    projection_matrix_2 = np.empty((3, 4))
    total_points_3d = np.zeros((1, 3))
    total_colors = np.zeros((1, 3))
    images = []

    # Get a list of images in the directory
    for img_filename in sorted(os.listdir(img_dir)):
        if '.jpg' in img_filename.lower() or '.png' in img_filename.lower():
            images += [img_filename]

    i = 0
    mesh = o3d.geometry.TriangleMesh.create_coordinate_frame()

    # Setting the reference two frames
    image_0 = cv2.imread(os.path.join(img_dir, images[i]))
    image_1 = cv2.imread(os.path.join(img_dir, images[i + 1]))

    # Initial features for the first two images
    pts0, pts1 = find_features(image_0, image_1)

    # Finding essential matrix
    essential_matrix, mask = cv2.findEssentialMat(pts0, pts1, intrinsic_matrix, method=cv2.RANSAC, prob=0.999, threshold=0.4, mask=None)
    pts0 = pts0[mask.ravel() == 1]
    pts1 = pts1[mask.ravel() == 1]

    # The pose obtained is for the second image with respect to the first image
    _, rotation_matrix, translation, mask = cv2.recoverPose(essential_matrix, pts0, pts1, intrinsic_matrix)  # finding the pose
    pts0 = pts0[mask.ravel() > 0]
    pts1 = pts1[mask.ravel() > 0]

    # Initial extrinsic matrix for the second image
    current_pose_matrix[:3, :3] = np.matmul(rotation_matrix, initial_pose_matrix[:3, :3])
    current_pose_matrix[:3, 3] = initial_pose_matrix[:3, 3] + np.matmul(initial_pose_matrix[:3, :3], translation.ravel())
    
    # Initial Projection Matrix for the second image
    projection_matrix_2 = np.matmul(intrinsic_matrix, current_pose_matrix)

    # Triangulate the first image pair and poses will be set as reference
    pts0, pts1, points_3d = Triangulation(projection_matrix_1, projection_matrix_2, pts0, pts1)

    # Reprojecting the 3D points on the image and calculating the reprojection error
    reprojection_error, points_3d, reprojection_points = compute_reprojection_error(points_3d, pts1, current_pose_matrix, intrinsic_matrix, homogeneity=1)
    print("REPROJECTION ERROR: ", reprojection_error)
    rotation, translation, pts1, points_3d, keypoints_0_transformed = PnP(points_3d, pts1, intrinsic_matrix, np.zeros((5, 1), dtype=np.float32), pts0, initial=1)

    rotation = np.eye(3)
    translation = np.array([[0], [0], [0]], dtype=np.float32)

    # Images to be taken into consideration
    total_images = len(images) - 2 

    # Store the projection matrices for the current image pair in the pose_array array
    pose_array = np.hstack((pose_array, projection_matrix_1.ravel()))
    pose_array = np.hstack((pose_array, projection_matrix_2.ravel()))

    # Gradient threshold for bundle adjustment optimization
    gradient_threshold = 0.4


    for image_index in tqdm(range(total_images)):
        # Acquire a new image to be added to the pipeline and acquire matches with image pair
        current_image = cv2.imread(os.path.join(img_dir, images[image_index + 2]))

        # Detect features in the new image (current_image) and find correspondences
        pts_, pts2  = find_features(image_1, current_image)

        # If not processing the first image pair, perform triangulation for 3D reconstruction
        if image_index != 0:
            # Triangulate 3D points using the previous camera poses (projection_matrix_1, projection_matrix_2) and correspondences (keypoints_previous, keypoints_current)
            pts0, pts1, points_3d = Triangulation(projection_matrix_1, projection_matrix_2, pts0, pts1)

            # Transpose the points matrix for further processing
            pts1 = pts1.T

            # Convert homogeneous coordinates of 3D points
            points_3d = cv2.convertPointsFromHomogeneous(points_3d.T)
            points_3d = points_3d[:, 0, :]

        # Data Association
        # Find common points between the 3D points from the previous pair and the new correspondences
        # Extract common points in keypoints_current and keypoints_previous
        indices_1, indices_2, temp_keypoints_current, temp_keypoints_new = data_association(pts1, pts_, pts2)

        common_keypoints_current_image = pts2[indices_2]
        common_keypoints_previous = pts_[indices_2]

        # We have the 3D - 2D Correspondence for the new image as well as the point cloud obtained from before. The common points can be used to find the world coordinates of the new image
        # using Perspective - n - Point (PnP)
        rotation, translation, common_keypoints_current_image, points_3d, common_keypoints_previous = PnP(points_3d[indices_1], common_keypoints_current_image, intrinsic_matrix, np.zeros((5, 1), dtype=np.float32), common_keypoints_previous, initial=0)

        # Find the equivalent projection matrix for the new image
        new_extrinsic_matrix = np.hstack((rotation, translation))
        new_projection_matrix = np.matmul(intrinsic_matrix, new_extrinsic_matrix)

        error, points_3d, _ = compute_reprojection_error(points_3d, common_keypoints_current_image, new_extrinsic_matrix, intrinsic_matrix, homogeneity=0)

        temp_keypoints_previous, temp_keypoints_current, points_3d = Triangulation(projection_matrix_2, new_projection_matrix, temp_keypoints_current, temp_keypoints_new)
        error, points_3d, _ = compute_reprojection_error(points_3d, temp_keypoints_current, new_extrinsic_matrix, intrinsic_matrix, homogeneity=1)
        print("Reprojection Error: ", error)

        # We are storing the pose for each image. This will be very useful during multiview stereo as this should be known
        pose_array = np.hstack((pose_array, new_projection_matrix.ravel()))

        # Bundle adjustment based on least squares
        if perform_bundle_adjustment:
            print("Bundle Adjustment...")
            points_3d, temp_keypoints_current, new_projection_matrix = bundle_adjustment(points_3d, temp_keypoints_current, new_projection_matrix, intrinsic_matrix, gradient_threshold)
            new_projection_matrix = np.matmul(intrinsic_matrix, new_projection_matrix)
            error, points_3d, _ = compute_reprojection_error(points_3d, temp_keypoints_current, new_extrinsic_matrix, intrinsic_matrix, homogeneity=0)
            print("Minimized error: ", error)
            total_points_3d = np.vstack((total_points_3d, points_3d))
            keypoints_current_reg = np.array(temp_keypoints_current, dtype=np.int32)
            colors = np.array([current_image[l[1], l[0]] for l in keypoints_current_reg])
            total_colors = np.vstack((total_colors, colors))
        else:
            total_points_3d = np.vstack((total_points_3d, points_3d[:, 0, :]))
            keypoints_current_reg = np.array(temp_keypoints_current, dtype=np.int32)
            colors = np.array([current_image[l[1], l[0]] for l in keypoints_current_reg.T])
            total_colors = np.vstack((total_colors, colors)) 

        projection_matrix_1 = np.copy(projection_matrix_2)

        fig.add_trace(go.Scatter(x=[i], y=[error], mode='markers'))
        fig.update_layout(title='Reprojection Error Over Iterations', xaxis_title='Iteration', yaxis_title='Reprojection Error')

        image_0 = np.copy(image_1)
        image_1 = np.copy(current_image)
        pts0 = np.copy(pts_)
        pts1 = np.copy(pts2)
        projection_matrix_2 = np.copy(new_projection_matrix)
        cv2.imshow('image', current_image)
        if cv2.waitKey(1) & 0xff == ord('q'):
            break

    fig.show()
    cv2.destroyAllWindows()

    # the obtained points cloud is registered and saved using open3d. It is saved in .ply form, which can be viewed using meshlab
    print("Processing Point Cloud...")
    create_pointcloud(path, total_points_3d, total_colors)
    print(f"Incremental SFM for {image_dataset} dataset completed!")
    # Saving projection matrices for all the images
    #np.savetxt('pose.csv', posearr, delimiter = '\n')


## main.py for execution of the above pipeline

- Input the Intrinsic Camera Matrx 
- Input opath for the dataset containing .jpg or .png images


In [11]:

# Input Camera Intrinsic Parameters
K = np.array([[2393.952166119461, -3.410605131648481e-13, 932.3821770809047],
              [0, 2398.118540286656, 628.2649953288065],
              [0, 0, 1]])

# Other parameters
bundle_adjustment = False
path = os.getcwd()
img_dir = os.path.join(path, 'vercingetorix')

sfm_pipeline(path, img_dir, K, bundle_adjustment)

Initialized Initial Parameters
REPROJECTION ERROR:  0.002249517015535686


  1%|▏         | 1/67 [00:01<01:09,  1.05s/it]

Reprojection Error:  0.3348880995189962


  3%|▎         | 2/67 [00:02<01:07,  1.04s/it]

Reprojection Error:  0.48364608675401205


...

 99%|█████████▊| 66/67 [01:06<00:00,  1.01it/s]

Reprojection Error:  0.687078574112956


100%|██████████| 67/67 [01:07<00:00,  1.01s/it]

Reprojection Error:  1.0290126583948747


Processing Point Cloud...
Incremental SFM for vercingetorix dataset completed!


In [21]:
cloud = o3d.io.read_point_cloud("Point_Cloud/sparse_ptcloud.ply") # Read point cloud
o3d.visualization.draw_geometries([cloud])    # Visualize point cloud      


### see the point cloud file sparse_ptcloud.ply in meshlab for a 3d point cloud of the 3d Reconstruction

![point cloud](pointcloud.png)
